In [1]:
import pandas as pd
import numpy as np
import math
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

In [2]:
df1 = pd.read_csv("train.csv")
df2 = pd.read_csv("test.csv")

In [3]:
pid = df2[['PID']].to_numpy()

In [4]:
df1 = df1.drop(['PID','Street', 'Utilities',  'Condition_2', 'Roof_Matl', 'Heating', 'Pool_QC', 'Misc_Feature', 'Low_Qual_Fin_SF', 'Pool_Area', 'Longitude','Latitude'], axis=1)
df2 = df2.drop(['PID','Street', 'Utilities',  'Condition_2', 'Roof_Matl', 'Heating', 'Pool_QC', 'Misc_Feature', 'Low_Qual_Fin_SF', 'Pool_Area', 'Longitude','Latitude'], axis=1)
df2

,MS_SubClass,MS_Zoning,Lot_Frontage,Lot_Area,Alley,Lot_Shape,Land_Contour,Lot_Config,Land_Slope,Neighborhood,...,Open_Porch_SF,Enclosed_Porch,Three_season_porch,Screen_Porch,Fence,Misc_Val,Mo_Sold,Year_Sold,Sale_Type,Sale_Condition
0,One_Story_1946_and_Newer_All_Styles,Residential_Low_Density,0,8926,No_Alley_Access,Slightly_Irregular,Lvl,Corner,Gtl,Edwards,...,0,0,0,160,Minimum_Privacy,0,10,2009,COD,Abnorml
1,Two_Story_1945_and_Older,Residential_Medium_Density,57,6876,No_Alley_Access,Regular,Lvl,Inside,Gtl,Iowa_DOT_and_Rail_Road,...,0,213,0,0,Minimum_Privacy,0,8,2009,WD,Normal
2,Two_Story_1946_and_Newer,Residential_Low_Density,0,13869,No_Alley_Access,Moderately_Irregular,Lvl,Corner,Gtl,Gilbert,...,75,0,0,0,No_Fence,0,7,2007,WD,Normal
3,One_Story_1946_and_Newer_All_Styles,Residential_Low_Density,87,10000,No_Alley_Access,Slightly_Irregular,Lvl,Corner,Gtl,North_Ames,...,0,0,0,385,No_Fence,0,2,2010,WD,Normal
4,One_and_Half_Story_Finished_All_Ages,Residential_Medium_Density,60,10320,Gravel,Regular,Lvl,Inside,Gtl,Iowa_DOT_and_Rail_Road,...,0,239,0,0,Minimum_Privacy,0,6,2008,WD,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
874,Two_Story_1946_and_Newer,Residential_Low_Density,80,11584,No_Alley_Access,Regular,Lvl,Inside,Gtl,Northwest_Ames,...,88,216,0,0,No_Fence,0,11,2007,WD,Normal
875,Two_Story_1946_and_Newer,Residential_Low_Density,72,7200,No_Alley_Access,Regular,Lvl,Inside,Gtl,Edwards,...,0,0,0,0,Minimum_Privacy,0,2,2008,WD,Normal
876,One_Story_1946_and_Newer_All_Styles,Residential_Low_Density,84,11670,No_Alley_Access,Slightly_Irregular,Lvl,Corner,Gtl,Somerset,...,191,0,0,0,No_Fence,0,3,2007,WD,Normal
877,Two_Story_PUD_1946_and_Newer,Floating_Village_Residential,30,3215,Paved,Regular,Lvl,Inside,Gtl,Somerset,...,172,0,0,0,No_Fence,0,4,2010,ConLD,Normal


In [5]:
winsor = ["Lot_Frontage", "Lot_Area", "Mas_Vnr_Area", "BsmtFin_SF_2", "Bsmt_Unf_SF", "Total_Bsmt_SF", "Second_Flr_SF", 'First_Flr_SF', "Gr_Liv_Area", "Garage_Area", "Wood_Deck_SF", "Open_Porch_SF", "Enclosed_Porch", "Three_season_porch", "Screen_Porch", "Misc_Val"]
for col in winsor:
    df1[col] = df1[col].clip(lower=0, upper=df1[col].quantile(0.95))
    df2[col] = df2[col].clip(lower=0, upper=df2[col].quantile(0.95))

In [6]:
y_train = np.log(df1[['Sale_Price']].values)
df1 = df1.drop(['Sale_Price'], axis=1)

In [7]:
df1.Garage_Yr_Blt.fillna(df1.Year_Built, inplace=True)
df2.Garage_Yr_Blt.fillna(df2.Year_Built, inplace=True)

In [8]:
cols=[]
columns = df1.columns.to_list()
categorical_feature_mask = df1.dtypes==object
categorical_cols = df1.columns[categorical_feature_mask].tolist()
for i,c in enumerate(columns):
    if c in categorical_cols:
        cols.append(i)

In [9]:
enc = ColumnTransformer(
    [('one_hot_encoder', OneHotEncoder(categories='auto', handle_unknown='ignore'), cols)],   # The column numbers to be transformed (here is [0] but can be [0, 1, 3])
    remainder='passthrough'                                         # Leave the rest of the columns untouched
)
enc.fit(df1)
X_train = enc.fit_transform(df1).toarray()
X_test = enc.transform(df2).toarray()

In [10]:
from sklearn.linear_model import Ridge
clf = Ridge(alpha=1)
clf.fit(X_train, y_train)

Ridge(alpha=1, copy_X=True, fit_intercept=True, max_iter=None, normalize=False,
      random_state=None, solver='auto', tol=0.001)

In [11]:
y_test = np.exp(clf.predict(X_test))

In [12]:
df3 = pd.read_csv("test_y.csv")

In [13]:
y_real = df3[['Sale_Price']].values

In [14]:
math.sqrt( (1/y_real.shape[0]) * np.sum((np.log(y_test) - np.log(y_real))**2))

0.12193873471131782

In [ ]:
#train accuracy

In [25]:
for i in y_test:
    print(i)

[73324.85796166]
[158623.61809577]
[182758.88809838]
[140838.63472688]
[140066.84192629]
[129467.61137172]
[134278.950924]
[189570.36004144]
[103019.46979775]
[324650.81048567]
[145744.09557182]
[145539.75594485]
[334342.51756721]
[165036.02756347]
[193176.12259124]
[137409.21308312]
[188902.95055882]
[107420.98777817]
[139967.73296936]
[138304.48798579]
[276732.14186893]
[169393.85576736]
[199387.23389179]
[152740.69113854]
[79866.76951513]
[78369.23506576]
[159382.92867475]
[135506.80385171]
[123865.5771554]
[226681.22345456]
[90600.08364497]
[122658.07126021]
[373169.33494288]
[176233.81056057]
[138369.8936217]
[145096.1398331]
[220013.28598969]
[123995.8983956]
[105197.08182775]
[246809.68983539]
[166969.96629131]
[290433.83721305]
[275111.63935735]
[337192.35138638]
[198274.41585988]
[100992.01221493]
[189921.96906897]
[133356.57259936]
[161157.28415202]
[110948.06517548]
[184794.02032773]
[173950.99760484]
[93044.22424228]
[83271.56209715]
[408765.54509052]
[211324.23666297]
[187

In [32]:
f = open("mysubmission1.txt", "w")
f.write("PID,  Sale_Price")
f.write("\n")
num = len(y_test)
for i,p in enumerate(y_test):
    send = str(pid[i][0])+", "+str(p[0])
    f.write(send)
    if i != num - 1:
        f.write("\n")
f.close()


In [15]:
from sklearn.ensemble import RandomForestRegressor
regr = RandomForestRegressor()
regr.fit(X_train, y_train)
y_test = np.exp(regr.predict(X_test))
#math.sqrt( (1/y_real.shape[0]) * np.sum((np.log(y_test) - np.log(y_real))**2))

C:\Program Files (x86)\Microsoft Visual Studio\Shared\Python36_64\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Program Files (x86)\Microsoft Visual Studio\Shared\Python36_64\lib\site-packages\ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until


In [16]:
math.sqrt( (1/y_real.shape[0]) * np.sum((np.log(y_test) - np.log(y_real))**2))

15.55829590841174

In [34]:
y_test

array([ 86592.65763184, 151273.77086648, 179516.73490604, 137823.08000249,
       136508.058464  , 112769.42383576, 138635.66232527, 186088.14473335,
        97683.22057218, 328226.71346786, 135700.75801727, 148164.61927309,
       322401.0551876 , 150690.67929766, 189048.08414916, 139295.35256537,
       179256.90617263, 102338.55777176, 142181.44543336, 110676.82621674,
       283502.08740454, 131196.86314134, 247221.57230098, 145735.3224943 ,
        82179.28842118,  81050.47907503, 147691.96976539, 129268.49852607,
       137959.96084008, 228806.46915175,  93188.16039303, 123363.81115681,
       363409.58929912, 149077.49538284, 137472.45133942, 145031.13460435,
       207227.39020295, 127206.75152317, 137018.77363394, 250180.67033765,
       169569.83600492, 248793.97145815, 268489.59071068, 348726.38027583,
       202559.83959138,  90745.40600563, 215581.35096503, 136563.87475483,
       165848.21382846, 124165.08793477, 164997.83164118, 173858.93749676,
        98377.93660995, 1

In [ ]:
f = open("mysubmission2.txt", "w")
f.write("PID,  Sale_Price")
f.write("\n")
num = len(y_test)
for i,p in enumerate(y_test):
    send = str(pid[i][0])+", "+str(p[0])
    f.write(send)
    if i != num - 1:
        f.write("\n")
f.close()